In [1]:
import pandas as pd
import numpy as np
from run import ManualTunning
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('data/creditcard.csv')

In [3]:
label = data.Class
data = data.drop(['Time', 'Class'], axis=1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data,label, test_size=0.33, random_state=42) 

In [5]:
lgb_params = {
             'n_jobs': 5, 'two_round': True, 'is_unbalance': True,
             'boosting_type': 'gbdt', 'learning_rate': 0.01, 'min_child_samples': 10,
             'min_child_weight': 5, 'n_estimators':1000, 'num_leaves':21,
             'max_depth': 6
}

### LightGBM Example

In [6]:
estimator = lgb.LGBMClassifier(**lgb_params, random_state = 10)

In [7]:
param_dict = {'n_estimators': [10, 100], 'max_depth': [4, 6]}
tunning = ManualTunning(clone(estimator), cv=3)

In [8]:
tunning.fit(X_train,y_train,param_dict)

[05/19 03:08:25] Initializing... Parameters: {'n_estimators': [10, 100], 'max_depth': [4, 6]}
[05/19 03:08:25] Models to be trained: 4
[05/19 03:08:25] Estimator: lightgbm.sklearn
[05/19 03:08:26] Parameter: {'n_estimators': 10, 'max_depth': 4} - Metric: auc - Value: 0.9474677526986658
[05/19 03:08:27] Parameter: {'n_estimators': 10, 'max_depth': 6} - Metric: auc - Value: 0.9530732968260113
[05/19 03:08:30] Parameter: {'n_estimators': 100, 'max_depth': 4} - Metric: auc - Value: 0.9614489946122249
[05/19 03:08:33] Parameter: {'n_estimators': 100, 'max_depth': 6} - Metric: auc - Value: 0.9661781313729239
[05/19 03:08:33] Best: [['max_depth', 6], ['n_estimators', 100]] - Score: 0.9661781313729239


In [9]:
for param, value in dict(tunning.scores_[0][0]).items():
   estimator.set_params(**{param:value})

In [10]:
estimator.fit(X_train, y_train)
print(f'ROCAUC - {roc_auc_score(y_test, estimator.predict_proba(X_test)[:, 1])}')

ROCAUC - 0.9765981812722798


### xgboost example

In [11]:
xgb_params = {
             'n_jobs': 5, 'two_round': True, 'is_unbalance': True,
             'boosting_type': 'gbdt', 'learning_rate': 0.01, 'min_child_samples': 10,
             'min_child_weight': 5, 'n_estimators':1000, 'num_leaves':21,
             'max_depth': 6
}

In [12]:
estimator = xgb.XGBClassifier(**xgb_params, random_state = 10)

In [13]:
param_dict = {'n_estimators': [10, 100], 'max_depth': [4, 6]}
tunning = ManualTunning(clone(estimator), cv=3)

In [14]:
tunning.fit(X_train, y_train,param_dict)

[05/19 03:08:35] Initializing... Parameters: {'n_estimators': [10, 100], 'max_depth': [4, 6]}
[05/19 03:08:35] Models to be trained: 4
[05/19 03:08:35] Estimator: xgboost.sklearn
[05/19 03:08:39] Parameter:{'n_estimators': 10, 'max_depth': 4} - Metric: auc - Value: 0.9108679077800005
[05/19 03:08:43] Parameter:{'n_estimators': 10, 'max_depth': 6} - Metric: auc - Value: 0.9108679077800005
[05/19 03:09:14] Parameter:{'n_estimators': 100, 'max_depth': 4} - Metric: auc - Value: 0.9152402389290968
[05/19 03:09:47] Parameter:{'n_estimators': 100, 'max_depth': 6} - Metric: auc - Value: 0.9152403112056127
[05/19 03:09:47] Best: [['max_depth', 6], ['n_estimators', 100]] - Score: 0.9152403112056127


In [15]:
for param, value in dict(tunning.scores_[0][0]).items():
   estimator.set_params(**{param:value})

In [16]:
estimator.fit(X_train, y_train)
print(f'ROCAUC - {roc_auc_score(y_test, estimator.predict_proba(X_test)[:, 1])}')

ROCAUC - 0.9327223369820128


### RandomForest example

In [17]:
estimator = RandomForestClassifier(random_state = 10)

In [18]:
param_dict = {'n_estimators': [10, 100], 'max_depth': [4, 6]}
tunning = ManualTunning(clone(estimator), cv=3)

In [19]:
tunning.fit(X_train, y_train,param_dict)

[05/19 03:10:03] Initializing... Parameters: {'n_estimators': [10, 100], 'max_depth': [4, 6]}
[05/19 03:10:03] Models to be trained: 4
[05/19 03:10:03] Estimator: sklearn.ensemble._forest
[05/19 03:10:10] Parameter: {'n_estimators': 10, 'max_depth': 4} - Metric: auc - Value: 0.9231969425498624
[05/19 03:10:20] Parameter: {'n_estimators': 10, 'max_depth': 6} - Metric: auc - Value: 0.948931615607406
[05/19 03:11:16] Parameter: {'n_estimators': 100, 'max_depth': 4} - Metric: auc - Value: 0.9594617453098889
[05/19 03:12:39] Parameter: {'n_estimators': 100, 'max_depth': 6} - Metric: auc - Value: 0.9647602857659244
[05/19 03:12:39] Best: [['max_depth', 6], ['n_estimators', 100]] - Score: 0.9647602857659244


In [20]:
for param, value in dict(tunning.scores_[0][0]).items():
   estimator.set_params(**{param:value})

In [21]:
estimator.fit(X_train, y_train)
print(f'ROCAUC - {roc_auc_score(y_test, estimator.predict_proba(X_test)[:, 1])}')

ROCAUC - 0.9748176959549449
